In [16]:
import os, datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image

time_str = (datetime.datetime.now()).strftime("%Y%m%d")
xml_dir = '/data/darknet/python/' + '20190111' + '_xml/'
image_dir = '/data/darknet/python/' + time_str + '_images_replaced_bkg/'
generated_image_dir = '/data/darknet/python/' + time_str + '_image_expand_obj/'
generated_xml_dir = '/data/darknet/python/' + time_str + '_xml_expand_obj/'

os.system('mkdir -p ' + xml_dir)
os.system('mkdir -p ' + image_dir)
os.system('mkdir -p ' + generated_image_dir)
os.system('mkdir -p ' + generated_xml_dir)

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
SEED_CHG_JPG = generated_image_dir
SEED_CHG_XML = generated_xml_dir

os.system("mkdir -p " + SEED_CHG_XML)
os.system("mkdir -p " + SEED_CHG_JPG)

import numpy as np
import cv2

In [17]:
seed_xml_names = os.listdir(SEED_XML_DIR)
#seed_img_names = os.listdir(SEED_IMG_DIR)
seed_xml_names.sort()
print(len(seed_xml_names))

101


In [22]:
x_shrink_scale = 0
y_shrink_scale = 0.2
padding = 127.5
for xml in seed_xml_names:
    
    # generate new jpg image file
    img_file = xml[:-4] + '.jpg'
    img_path = SEED_IMG_DIR + img_file
    img = cv2.imread(img_path)
    array = np.zeros_like(img)
    array.fill(padding)   
    new_w = int(img.shape[1]*(1-x_shrink_scale))
    new_h = int(img.shape[0]*(1-y_shrink_scale))
    shrink_width = img.shape[1] - new_w
    shrink_heigt = img.shape[0] - new_h
    #img = cv2.resize(img, (new_w, img.shape[0]))
    #array[:, int(shrink_width/2):int(shrink_width/2) + new_w] = img
    img = cv2.resize(img, (new_w, new_h))
    array[int(shrink_heigt/2):int(shrink_heigt/2) + new_h, int(shrink_width/2):int(shrink_width/2) + new_w] = img
    cv2.imwrite(SEED_CHG_JPG + "ge_shrink_scale_" + str(x_shrink_scale) + "_" + str(y_shrink_scale) + "_" + str(padding) + "_"+ img_file, array)
    
    # generate new anotation xml file    
    xml_path = SEED_XML_DIR + xml
    in_file = open(xml_path)
    tree=ET.parse(in_file)
    root = tree.getroot()
    
    for obj in root.iter('object'):
        '''
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        '''
        xmlbox = obj.find('bndbox')
        xmin, xmax = int(xmlbox.find('xmin').text), int(xmlbox.find('xmax').text)
        ymin, ymax = int(xmlbox.find('ymin').text), int(xmlbox.find('ymax').text)
        xmlbox.find('xmin').text = str(int(xmin*(1-x_shrink_scale) + shrink_width/2))
        xmlbox.find('xmax').text = str(int(xmax*(1-x_shrink_scale) + shrink_width/2))
        xmlbox.find('ymin').text = str(int(ymin*(1-y_shrink_scale) + shrink_heigt/2))
        xmlbox.find('ymax').text = str(int(ymax*(1-y_shrink_scale) + shrink_heigt/2))
    tree.write(SEED_CHG_XML + "ge_shrink_scale_" + str(x_shrink_scale) + "_" + str(y_shrink_scale) +  "_" + str(padding) +"_" + xml)
    #cv2.imshow("demo", array)
    #cv2.waitKey(5000)
    #break